In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

In [2]:
# 1. Prepare Data (Integer Encoded + Padded)
# Shape: (3 rows, 4 timesteps, 1 feature)
# We must reshape it because RNN expects 3D input: (Batch, Steps, Features)
X_integer = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [5, 9, 10, 0]
])

In [3]:
X_integer

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 5,  9, 10,  0]])

In [4]:
X_integer.reshape(3,4,1)

array([[[ 1],
        [ 2],
        [ 3],
        [ 4]],

       [[ 5],
        [ 6],
        [ 7],
        [ 8]],

       [[ 5],
        [ 9],
        [10],
        [ 0]]])

In [5]:
# 2. Build Model
model = Sequential()
# Input shape says: "4 steps long, 1 number at each step"
model.add(SimpleRNN(32, input_shape=(4, 1))) 
model.add(Dense(1, activation='sigmoid'))

2026-01-22 12:47:47.480583: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2026-01-22 12:47:47.480652: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2026-01-22 12:47:47.480656: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2026-01-22 12:47:47.480716: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-01-22 12:47:47.480762: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer.

In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn (SimpleRNN)          │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,121 (4.38 KB)

 Trainable params: 1,121 (4.38 KB)

 Non-trainable params: 0 (0.00 B)

# Embedding 

In [9]:
from tensorflow.keras.layers import Embedding

# 1. Prepare Data (Just 2D array is fine here!)
X_embed = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [5, 9, 10, 0]
])

# Creating dummy targets (Labels) for training
# 0 = Negative Review, 1 = Positive Review
y_train = np.array([0, 1, 0])

In [11]:
# 2. Build Model
model = Sequential()

# Layer 1: Embedding
# vocabulary_size=11 (0 to 10), vector_dim=5 (each word becomes vector of size 5)
# input_length=4 (timesteps)
model.add(Embedding(input_dim=11, output_dim=5, input_length=4))

# VISUALIZATION:
# Input: [2]
# Embedding Layer Output: [0.1, 0.9, -0.2, 0.5, 0.1] (Vector of size 5)
# This Vector of size 5 goes into the RNN!

model.add(SimpleRNN(32)) 
model.add(Dense(1, activation='sigmoid'))

In [12]:
# --- 3. COMPILE (The Missing Part) ---
# optimizer='adam': The standard choice (adjusts learning rate automatically)
# loss='binary_crossentropy': Because we are doing binary classification (0 or 1)
# metrics=['accuracy']: To see how correct we are
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [13]:
# --- 4. TRAIN (Fit) ---
# epochs=10: Go through the data 10 times
print("Training Shuru! 🚀")
history = model.fit(X_embed, y_train, epochs=10, verbose=1)

Training Shuru! 🚀
Epoch 1/10


2026-01-22 12:53:43.550595: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.3333 - loss: 0.6826
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step - accuracy: 0.6667 - loss: 0.6773
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.6667 - loss: 0.6718
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6667 - loss: 0.6662
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.6667 - loss: 0.6604
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6667 - loss: 0.6543
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.6667 - loss: 0.6481
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.6667 - loss: 0.6417
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.6667 - loss: 0.6350
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6667 - loss: 0.6280


In [14]:
# --- 5. PREDICT (Test) ---
print("\nPrediction on first sample:")
pred = model.predict(X_embed[0].reshape(1,4))
print(f"Raw Output: {pred}")
print(f"Class: {'Positive' if pred > 0.5 else 'Negative'}")


Prediction on first sample:
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 679ms/step
Raw Output: [[0.44407088]]
Class: Negative
